In [4]:
import requests
import pandas as pd
import os

pd.options.display.max_columns=100

In [8]:
# from https://stackoverflow.com/questions/16694907/download-large-file-in-python-with-requests
def download_file(url, local_path):
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
    print('Written to', local_path)

In [9]:
"""
FEC bulk data files

https://www.fec.gov/data/browse-data/?tab=bulk-data

"""

def get_mt_federal_candidates():
    url = 'https://www.fec.gov/files/bulk-downloads/2020/cn20.zip'
    local_path = 'data/candidates20.zip'
    download_file(url, local_path)
    names = ["CAND_ID", "CAND_NAME", "CAND_PTY_AFFILIATION", "CAND_ELECTION_YR", "CAND_OFFICE_ST", "CAND_OFFICE", "CAND_OFFICE_DISTRICT", "CAND_ICI", "CAND_STATUS", "CAND_PCC", "CAND_ST1", "CAND_ST2", "CAND_CITY", "CAND_ST", "CAND_ZIP"]
    df = pd.read_csv(local_path, delimiter="|", header=None, names=names)
    mt_2020 = df[
        (df['CAND_OFFICE_ST'] == 'MT') 
        & (df['CAND_ELECTION_YR'] == 2020)
        & (df['CAND_OFFICE'].isin(['H','S'])) # House or Senate
    ]
    mt_2020.to_json('data/mt-2020-candidates.json', orient='records')
    return mt_2020
    
mt_2020 = get_mt_federal_candidates()

Written to data/candidates20.zip


In [40]:
def get_campaign_summaries():
    # Summary for current campaigns
    # documentation: https://www.fec.gov/campaign-finance-data/all-candidates-file-description/
    url = 'https://www.fec.gov/files/bulk-downloads/2020/webl20.zip'
    local_path = 'data/current20.zip'
    names = ['CAND_ID', 'CAND_NAME', 'CAND_ICI', 'PTY_CD', 'CAND_PTY_AFFILIATION', 'TTL_RECEIPTS', 'TRANS_FROM_AUTH',
            'TTL_DISB', 'TRANS_TO_AUTH', 'COH_BOP', 'COH_COP', 'CAND_CONTRIB', 'CAND_LOANS', 'OTHER_LOANS', 
             'CAND_LOAN_REPAY', 'OTHER_LOAN_REPAY', 'DEBTS_OWED_BY', 'TTL_INDIV_CONTRIB', 
             'CAND_OFFICE_ST', 'CAND_OFFICE_DISTRICT', 'SPEC_ELECTION', 'PRIM_ELECTION', 'RUN_ELECTION', 'GEN_ELECTION',
             'GEN_ELECTION_PRECENT', 'OTHER_POL_CMTE_CONTRIB', 'POL_PTY_CONTRIB', 'CVG_END_DT', 'INDIV_REFUNDS', 'CMTE_REFUNDS'
            ]
    download_file(url, local_path)
    current = pd.read_csv(local_path, delimiter="|", header=None, names=names)
    current = current[current['CAND_OFFICE_ST'] == 'MT']
    current.drop(labels=['SPEC_ELECTION','PRIM_ELECTION','RUN_ELECTION','GEN_ELECTION','GEN_ELECTION_PRECENT'], axis=1, inplace=True)
    return current

current = get_campaign_summaries()

Written to data/current20.zip


In [42]:
current

,CAND_ID,CAND_NAME,CAND_ICI,PTY_CD,CAND_PTY_AFFILIATION,TTL_RECEIPTS,TRANS_FROM_AUTH,TTL_DISB,TRANS_TO_AUTH,COH_BOP,COH_COP,CAND_CONTRIB,CAND_LOANS,OTHER_LOANS,CAND_LOAN_REPAY,OTHER_LOAN_REPAY,DEBTS_OWED_BY,TTL_INDIV_CONTRIB,CAND_OFFICE_ST,CAND_OFFICE_DISTRICT,OTHER_POL_CMTE_CONTRIB,POL_PTY_CONTRIB,CVG_END_DT,INDIV_REFUNDS,CMTE_REFUNDS
459,H0MT00090,"WINTER, THOMAS",C,1,DEM,133635.00,0.00,29789.93,0.00,0.00,103845.07,234.00,0.0,0.0,0.0,0.0,0.0,128401.00,MT,0,5000.00,0.0,06/30/2019,0.00,0.00
460,H8MT01232,"WILLIAMS, KATHLEEN",C,1,DEM,446342.79,35820.26,133214.35,35820.26,36957.25,350085.69,0.00,0.0,0.0,0.0,0.0,0.0,394967.53,MT,0,0.00,0.0,07/31/2019,186.00,0.00
461,H4MT00050,"ROSENDALE, MATT MR.",C,2,REP,269179.17,97.18,40441.05,0.00,76178.38,304916.50,0.00,0.0,0.0,0.0,0.0,157831.0,192418.56,MT,0,6500.00,0.0,06/30/2019,2700.00,0.00
462,H4MT01033,"STAPLETON, COREY",C,2,REP,108014.00,0.00,25.00,0.00,0.00,107989.00,0.00,0.0,0.0,0.0,0.0,0.0,108014.00,MT,0,0.00,0.0,06/30/2019,0.00,0.00
463,H8MT01182,"GIANFORTE, GREG",I,2,REP,357457.35,72825.36,322806.70,0.00,31079.69,65730.34,0.00,0.0,0.0,0.0,0.0,500000.0,67430.36,MT,0,131475.00,50.0,06/30/2019,1162.10,25597.79
1098,S0MT00090,"COLLINS, WILMOT JAMES",C,1,DEM,91764.37,0.00,39138.31,0.00,0.00,52626.06,1064.86,0.0,0.0,0.0,0.0,0.0,90699.51,MT,0,0.00,0.0,06/30/2019,0.00,0.00
1099,S2MT00096,"DAINES, STEVEN",I,2,REP,2641188.43,422388.20,581045.41,0.00,1450176.34,3510319.36,0.00,0.0,0.0,0.0,0.0,0.0,1417549.82,MT,0,794692.49,0.0,06/30/2019,15472.75,1200.00


In [10]:
def get_individual_ccontributions():
    # bulk download individual contributions
    url = 'https://www.fec.gov/files/bulk-downloads/2020/indiv20.zip'
    local_path = 'data/individual-contributions-20.zip'
    if not os.path.exists(local_path): download_file(url, local_path) # for testing
    names = ["CMTE_ID", "AMNDT_IND", "RPT_TP", "TRANSACTION_PGI", "IMAGE_NUM", "TRANSACTION_TP", "ENTITY_TP", "NAME", "CITY", "STATE", "ZIP_CODE", "EMPLOYER", "OCCUPATION", "TRANSACTION_DT", "TRANSACTION_AMT", "OTHER_ID", "TRAN_ID", "FILE_NUM", "MEMO_CD", "MEMO_TEXT", "SUB_ID"]
    individual = pd.read_csv('./data/individual-contributions-20/itcont.txt', delimiter="|", header=None, names=names)
    mt_committees = list(mt_2020['CAND_PCC'])
    mt_indv = individual[individual['CMTE_ID'].isin(mt_committees)].copy()
    mt_indv = mt_indv.merge(mt_2020[['CAND_PCC','CAND_NAME']], left_on='CMTE_ID', right_on='CAND_PCC')
    return mt_indv

In [11]:
mt_indv = get_individual_ccontributions()

In [12]:
mt_indv.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,CAND_PCC,CAND_NAME
0,C00491357,N,Q1,P,201904159146337909,15,IND,"HOPEMAN, BERTRAM",BOZEMAN,MT,597711860,SHERBERT ENTERPRISES,OWNER,2062019,250,NaN,A3EBB07625665429FA35,1326203,NaN,NaN,4041820191646715471,C00491357,"DAINES, STEVEN"
1,C00491357,N,Q1,P,201904159146337910,15,IND,"ROGERS, NATALIE",WASHINGTON,DC,200034538,SUBJECT MATTER,CONSULTANT,3132019,1000,NaN,A05D62BAB38D04532B75,1326203,NaN,NaN,4041820191646715473,C00491357,"DAINES, STEVEN"
2,C00491357,N,Q1,P,201904159146337910,15,IND,"BRUEGGEMAN, JOHN",OLYMPIA,WA,985012228,MVSC,EXECUTIVE,3152019,2800,NaN,A05FD37296E394B978F9,1326203,NaN,NaN,4041820191646715475,C00491357,"DAINES, STEVEN"
3,C00491357,N,Q1,P,201904159146337910,15,IND,"BILLION, JOE",BOZEMAN,MT,597188706,BILLION AUTO SALES,OWNER,2062019,2700,NaN,ADE03926611B7443C882,1326203,NaN,NaN,4041820191646715477,C00491357,"DAINES, STEVEN"
4,C00491357,N,Q1,P,201904159146337911,15,IND,"KING, HARVEY",KAILUA,HI,967343118,NONE,RETIRED,3212019,100,NaN,A53C2A6EA38864360B7B,1326203,NaN,NaN,4041820191646715479,C00491357,"DAINES, STEVEN"


In [33]:
mt_indv.groupby('CAND_NAME').agg({'TRANSACTION_AMT': sum})

,TRANSACTION_AMT
CAND_NAME,
"COLLINS, WILMOT JAMES",51449
"DAINES, STEVEN",1224141
"GIANFORTE, GREG",50620
"ROSENDALE, MATT MR.",192234
"STAPLETON, COREY",105910
"WILLIAMS, KATHLEEN",259505
"WINTER, THOMAS",105669


In [34]:
mt_2020

,CAND_ID,CAND_NAME,CAND_PTY_AFFILIATION,CAND_ELECTION_YR,CAND_OFFICE_ST,CAND_OFFICE,CAND_OFFICE_DISTRICT,CAND_ICI,CAND_STATUS,CAND_PCC,CAND_ST1,CAND_ST2,CAND_CITY,CAND_ST,CAND_ZIP
525,H0MT00090,"WINTER, THOMAS",DEM,2020,MT,H,0.0,C,C,C00700963,P.O. BOX 8248,NaN,MISSOULA,MT,59807.0
526,H0MT00108,"DOOLING, JOE",REP,2020,MT,H,0.0,C,N,C00709923,3855 CEDAR VALLEY RD,NaN,HELENA,MT,59602.0
527,H0MT00116,"RAINS, MATTHEW",DEM,2020,MT,H,0.0,C,N,C00710509,PO BOX 1710,NaN,GREAT FALLS,MT,59403.0
528,H0MT01080,"JOHNSON, TIMOTHY ALAN",REP,2020,MT,H,0.0,C,N,C00711002,PO BOX 1088,NaN,CORVALLIS,MT,59828.0
529,H0MT01098,"LAMM, DEBRA",REP,2020,MT,H,0.0,C,N,C00718585,PO BOX 1915,NaN,LIVINGSTON,MT,59047.0
1388,H4MT00050,"ROSENDALE, MATT MR.",REP,2020,MT,H,0.0,C,N,C00548289,PO BOX 4907,NaN,HELENA,MT,596044907.0
1389,H4MT01033,"STAPLETON, COREY",REP,2020,MT,H,0.0,C,C,C00709915,2015 EASTRIDGE DR,NaN,BILLINGS,MT,591027904.0
2585,H8MT01182,"GIANFORTE, GREG",REP,2020,MT,H,0.0,I,C,C00631945,1320 MANLEY RD,NaN,BOZEMAN,MT,597158779.0
2586,H8MT01232,"WILLIAMS, KATHLEEN",DEM,2020,MT,H,0.0,C,C,C00701748,PO BOX 548,NaN,BOZEMAN,MT,59771.0
4565,S0MT00090,"COLLINS, WILMOT JAMES",DEM,2020,MT,S,0.0,C,C,C00706317,PO BOX 458,NaN,HELENA,MT,59624.0


In [ ]:
# # Current campaigns
# # documentation: https://www.fec.gov/campaign-finance-data/all-candidates-file-description/


# url = 'https://www.fec.gov/files/bulk-downloads/2020/weball20.zip'
# local_path = 'data/summary20.zip'
# names = ['CAND_ID', 'CAND_NAME', 'CAND_ICI', 'PTY_CD', 'CAND_PTY_AFFILIATION', 'TTL_RECEIPTS', 'TRANS_FROM_AUTH',
#         'TTL_DISB', 'TRANS_TO_AUTH', 'COH_BOP', 'COH_COP', 'CAND_CONTRIB', 'CAND_LOANS', 'OTHER_LOANS', 
#          'CAND_LOAN_REPAY', 'OTHER_LOAN_REPAY', 'DEBTS_OWED_BY', 'TTL_INDIV_CONTRIB', 
#          'CAND_OFFICE_ST', 'CAND_OFFICE_DISTRICT', 'SPEC_ELECTION', 'PRIM_ELECTION', 'RUN_ELECTION', 'GEN_ELECTION',
#          'GEN_ELECTION_PRECENT', 'OTHER_POL_CMTE_CONTRIB', 'POL_PTY_CONTRIB', 'CVG_END_DT', 'INDIV_REFUNDS', 'CMTE_REFUNDS'
#         ]
# download_file(url, local_path)
# df = pd.read_csv(local_path, delimiter="|", header=None, names=names)

In [49]:
# df = pd.read_csv(local_path, delimiter="|", header=None, names=names)
# mt = df[
#     (df['CAND_OFFICE_ST'] == 'MT') 
# #         & (df['CAND_ELECTION_YR'] == 2020)
# #         & (df['CAND_OFFICE'].isin(['H','S'])) # House or Senate
# ]

In [48]:
mt

,CAND_ID,CAND_NAME,CAND_ICI,PTY_CD,CAND_PTY_AFFILIATION,TTL_RECEIPTS,TRANS_FROM_AUTH,TTL_DISB,TRANS_TO_AUTH,COH_BOP,COH_COP,CAND_CONTRIB,CAND_LOANS,OTHER_LOANS,CAND_LOAN_REPAY,OTHER_LOAN_REPAY,DEBTS_OWED_BY,TTL_INDIV_CONTRIB,CAND_OFFICE_ST,CAND_OFFICE_DISTRICT,SPEC_ELECTION,PRIM_ELECTION,RUN_ELECTION,GEN_ELECTION,GEN_ELECTION_PRECENT,OTHER_POL_CMTE_CONTRIB,POL_PTY_CONTRIB,CVG_END_DT,INDIV_REFUNDS,CMTE_REFUNDS
1002,H0MT00090,"WINTER, THOMAS",C,1,DEM,133635.00,0.00,29789.93,0.00,0.00,103845.07,234.00,0.0,0.0,0.0,0.0,0.00,128401.00,MT,0,NaN,NaN,NaN,NaN,NaN,5000.00,0.0,06/30/2019,0.00,0.00
1003,H8MT01232,"WILLIAMS, KATHLEEN",C,1,DEM,446342.79,35820.26,133214.35,35820.26,36957.25,350085.69,0.00,0.0,0.0,0.0,0.0,0.00,394967.53,MT,0,NaN,NaN,NaN,NaN,NaN,0.00,0.0,07/31/2019,186.00,0.00
1004,H2MT01060,"DAINES, STEVEN",C,2,REP,2641188.43,422388.20,581045.41,0.00,1450176.34,3510319.36,0.00,0.0,0.0,0.0,0.0,0.00,1417549.82,MT,0,NaN,NaN,NaN,NaN,NaN,794692.49,0.0,06/30/2019,15472.75,1200.00
1005,H4MT00050,"ROSENDALE, MATT MR.",C,2,REP,269179.17,97.18,40441.05,0.00,76178.38,304916.50,0.00,0.0,0.0,0.0,0.0,157831.00,192418.56,MT,0,NaN,NaN,NaN,NaN,NaN,6500.00,0.0,06/30/2019,2700.00,0.00
1006,H4MT01033,"STAPLETON, COREY",C,2,REP,108014.00,0.00,25.00,0.00,0.00,107989.00,0.00,0.0,0.0,0.0,0.0,0.00,108014.00,MT,0,NaN,NaN,NaN,NaN,NaN,0.00,0.0,06/30/2019,0.00,0.00
1007,H4MT01041,"ZINKE, RYAN K",I,2,REP,0.00,0.00,648.55,0.00,12983.47,12334.92,0.00,0.0,0.0,0.0,0.0,0.00,0.00,MT,0,NaN,NaN,NaN,NaN,NaN,0.00,0.0,06/30/2019,0.00,0.00
1008,H8MT01182,"GIANFORTE, GREG",I,2,REP,357457.35,72825.36,322806.70,0.00,31079.69,65730.34,0.00,0.0,0.0,0.0,0.0,500000.00,67430.36,MT,0,NaN,NaN,NaN,NaN,NaN,131475.00,50.0,06/30/2019,1162.10,25597.79
2302,S0MT00090,"COLLINS, WILMOT JAMES",C,1,DEM,91764.37,0.00,39138.31,0.00,0.00,52626.06,1064.86,0.0,0.0,0.0,0.0,0.00,90699.51,MT,0,NaN,NaN,NaN,NaN,NaN,0.00,0.0,06/30/2019,0.00,0.00
2303,S6MT00162,"TESTER, R. JON",I,1,DEM,255735.76,0.00,337626.71,0.00,480573.87,398682.92,0.00,0.0,0.0,0.0,0.0,0.00,101099.66,MT,0,NaN,NaN,NaN,NaN,NaN,80000.00,0.0,06/30/2019,-3605.09,0.00
2304,S8MT00010,"BAUCUS, MAX",C,1,DEM,6086.51,0.00,14873.15,5000.00,943885.05,935098.41,0.00,0.0,0.0,0.0,0.0,0.00,0.00,MT,0,NaN,NaN,NaN,NaN,NaN,0.00,0.0,06/30/2019,0.00,0.00


In [32]:
mt_2020

,CAND_ID,CAND_NAME,CAND_PTY_AFFILIATION,CAND_ELECTION_YR,CAND_OFFICE_ST,CAND_OFFICE,CAND_OFFICE_DISTRICT,CAND_ICI,CAND_STATUS,CAND_PCC,CAND_ST1,CAND_ST2,CAND_CITY,CAND_ST,CAND_ZIP
525,H0MT00090,"WINTER, THOMAS",DEM,2020,MT,H,0.0,C,C,C00700963,P.O. BOX 8248,NaN,MISSOULA,MT,59807.0
526,H0MT00108,"DOOLING, JOE",REP,2020,MT,H,0.0,C,N,C00709923,3855 CEDAR VALLEY RD,NaN,HELENA,MT,59602.0
527,H0MT00116,"RAINS, MATTHEW",DEM,2020,MT,H,0.0,C,N,C00710509,PO BOX 1710,NaN,GREAT FALLS,MT,59403.0
528,H0MT01080,"JOHNSON, TIMOTHY ALAN",REP,2020,MT,H,0.0,C,N,C00711002,PO BOX 1088,NaN,CORVALLIS,MT,59828.0
529,H0MT01098,"LAMM, DEBRA",REP,2020,MT,H,0.0,C,N,C00718585,PO BOX 1915,NaN,LIVINGSTON,MT,59047.0
1388,H4MT00050,"ROSENDALE, MATT MR.",REP,2020,MT,H,0.0,C,N,C00548289,PO BOX 4907,NaN,HELENA,MT,596044907.0
1389,H4MT01033,"STAPLETON, COREY",REP,2020,MT,H,0.0,C,C,C00709915,2015 EASTRIDGE DR,NaN,BILLINGS,MT,591027904.0
2585,H8MT01182,"GIANFORTE, GREG",REP,2020,MT,H,0.0,I,C,C00631945,1320 MANLEY RD,NaN,BOZEMAN,MT,597158779.0
2586,H8MT01232,"WILLIAMS, KATHLEEN",DEM,2020,MT,H,0.0,C,C,C00701748,PO BOX 548,NaN,BOZEMAN,MT,59771.0
4565,S0MT00090,"COLLINS, WILMOT JAMES",DEM,2020,MT,S,0.0,C,C,C00706317,PO BOX 458,NaN,HELENA,MT,59624.0
